In [1]:
import os
import json
import pandas as pd

transcriptions_folder = "D:/Personal Projects/losse_talk/src/dataset/transcriptions" 

In [2]:
import re

def extract_episode_number(filename):
    match = re.search(r'(?:Episode_|Ep_)(\d+)', filename, re.IGNORECASE)  
    if match:
        return int(match.group(1))  # Convert to integer for easy sorting/comparison
    return None

In [3]:
data = []

for file_name in os.listdir(transcriptions_folder):
    ep_no = extract_episode_number(file_name)
    file_path = os.path.join(transcriptions_folder, file_name)
    with open(file_path, 'r') as f:
        transcription = json.load(f)

    line = ""
    speaker = "speaker_0"
    for word in transcription['words']:
        if word['text'][0] == "(" and word['text'][-1] == ")":
            continue
        
        if word['speaker_id'] == speaker:
            line += word['text']
        else:
            data.append({
                # "index": 1, 
                "episode no": ep_no, 
                "name": "moeen" if speaker=="speaker_1" else "anwar",
                "line": line    
            })
            speaker = word['speaker_id']
            line = word['text']

In [4]:
df = pd.DataFrame(data)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2350 entries, 0 to 2349
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   episode no  2350 non-null   int64 
 1   name        2350 non-null   object
 2   line        2350 non-null   object
dtypes: int64(1), object(2)
memory usage: 55.2+ KB


In [6]:
df.to_csv("srcipt.csv", index=True, index_label="index")

In [7]:
from tqdm.auto import tqdm

In [19]:
SYSTEM_PROMPT = """You are Moeen, the razor-sharp mimic artist. 
You combine incisive wit with candid honesty, blending dark humor and practical insights. 
Never shy away from delivering a truth that cuts deep."""

new_rows = []
for i in tqdm(range(len(data) - 1)):
    current_row = data[i]
    next_row = data[i + 1]

    if current_row["name"] != 'moeen' and next_row["name"] == 'moeen':
        if current_row["episode no"] == next_row["episode no"]:
            new_rows.append({
                "conversations": [
                    {"from": "system", "value": SYSTEM_PROMPT.strip()},
                    {"from": "human", "value": current_row["line"].strip()},
                    {"from": "gpt", "value": next_row["line"].strip()}
                ]
            })


  0%|          | 0/2349 [00:00<?, ?it/s]

In [20]:
new_rows[0]

{'conversations': [{'from': 'system',
   'value': 'You are Moeen, the razor-sharp mimic artist. \nYou combine incisive wit with candid honesty, blending dark humor and practical insights. \nNever shy away from delivering a truth that cuts deep.'},
  {'from': 'human',
   'value': 'السلام علیکم، دوست ٹاک کے ساتھ انور مقصود آپ کی خدمت میں。 دوست ٹاک میں جو آج میرے مہمان ہیں وہ ہارمونیم نواز ہیں۔'},
  {'from': 'gpt', 'value': 'سنئے فرندز کم، السلام علیکم۔'}]}

In [21]:
from datasets import Dataset
sharegpt_dataset = Dataset.from_list(new_rows)

In [23]:
from huggingface_hub import notebook_login

notebook_login()

In [24]:
sharegpt_dataset.push_to_hub("ussmaanaali/loose-talk-transcripts-sharegpt")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/351 [00:00<?, ?B/s]

c:\Users\Usman\anaconda3\envs\seo\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Usman\.cache\huggingface\hub\datasets--ussmaanaali--loose-talk-transcripts-sharegpt. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/datasets/ussmaanaali/loose-talk-transcripts-sharegpt/commit/40ee72a9f3844b364eb5a98029a13ce4b4dbdfca', commit_message='Upload dataset', commit_description='', oid='40ee72a9f3844b364eb5a98029a13ce4b4dbdfca', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/ussmaanaali/loose-talk-transcripts-sharegpt', endpoint='https://huggingface.co', repo_type='dataset', repo_id='ussmaanaali/loose-talk-transcripts-sharegpt'), pr_revision=None, pr_num=None)